In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
np.random.seed(42)

In [6]:
email_table = pd.read_csv('email_table.csv')
email_opened_table = pd.read_csv('email_opened_table.csv')
link_clicked_table = pd.read_csv('link_clicked_table.csv')

In [7]:
print("Email table shape:", email_table.shape)
print("Email opened table shape:", email_opened_table.shape)
print("Link clicked table shape:", link_clicked_table.shape)

Email table shape: (100000, 7)
Email opened table shape: (10345, 1)
Link clicked table shape: (2119, 1)


In [8]:
print(email_table.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   email_id             100000 non-null  int64 
 1   email_text           100000 non-null  object
 2   email_version        100000 non-null  object
 3   hour                 100000 non-null  int64 
 4   weekday              100000 non-null  object
 5   user_country         100000 non-null  object
 6   user_past_purchases  100000 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 5.3+ MB
None


In [9]:
print(email_table.isnull().sum())

email_id               0
email_text             0
email_version          0
hour                   0
weekday                0
user_country           0
user_past_purchases    0
dtype: int64


In [10]:
email_table.columns = email_table.columns.str.strip('"')

In [11]:
email_table['opened'] = email_table['email_id'].isin(email_opened_table['email_id']).astype(int)
email_table['clicked'] = email_table['email_id'].isin(link_clicked_table['email_id']).astype(int)

In [12]:
email_table['is_weekend'] = email_table['weekday'].isin(['Saturday', 'Sunday'])
email_table['hour_bucket'] = pd.cut(email_table['hour'], bins=[0,6,12,18,24], labels=['Night', 'Morning', 'Afternoon', 'Evening'])

In [14]:
total_emails = len(email_table)
opened_emails = email_table['opened'].sum()
clicked_emails = email_table['clicked'].sum()

open_rate = opened_emails / total_emails * 100
click_rate = clicked_emails / total_emails * 100
click_to_open_rate = clicked_emails / opened_emails * 100 if opened_emails > 0 else 0

print(f"\nEmail Campaign Performance:")
print(f"Total emails sent: {total_emails}")
print(f"Number of emails opened: {opened_emails} ({open_rate:.2f}%)")
print(f"Number of links clicked: {clicked_emails} ({click_rate:.2f}%)")
print(f"Click-to-open rate: {click_to_open_rate:.2f}%")


Email Campaign Performance:
Total emails sent: 100000
Number of emails opened: 10345 (10.35%)
Number of links clicked: 2119 (2.12%)
Click-to-open rate: 20.48%


In [15]:
df_analysis = email_table.copy()

#categorical variables to category type
categorical_cols = ['email_text', 'email_version', 'weekday', 'user_country']
for col in categorical_cols:
    df_analysis[col] = df_analysis[col].astype('category')

#Basic statS
print("\nSummary Statistics:")
print(df_analysis.describe(include='all'))


Summary Statistics:
             email_id  email_text email_version           hour   weekday  \
count   100000.000000      100000        100000  100000.000000    100000   
unique            NaN           2             2            NaN         7   
top               NaN  long_email       generic            NaN  Saturday   
freq              NaN       50276         50209            NaN     14569   
mean    498690.196160         NaN           NaN       9.059300       NaN   
std     289230.727534         NaN           NaN       4.439637       NaN   
min          8.000000         NaN           NaN       1.000000       NaN   
25%     246708.250000         NaN           NaN       6.000000       NaN   
50%     498447.000000         NaN           NaN       9.000000       NaN   
75%     749942.750000         NaN           NaN      12.000000       NaN   
max     999998.000000         NaN           NaN      24.000000       NaN   

       user_country  user_past_purchases         opened        cli

# **Visualize**

In [27]:
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
sns.barplot(x='email_text', y='opened', data=df_analysis)
plt.title('Open Rate by Email Text Length')
plt.ylabel('Open Rate')

plt.subplot(2, 3, 2)
sns.barplot(x='email_text', y='clicked', data=df_analysis)
plt.title('Click Rate by Email Text Length')
plt.ylabel('Click Rate')

plt.subplot(2, 3, 3)
sns.barplot(x='email_version', y='opened', data=df_analysis)
plt.title('Open Rate by Email Version')
plt.ylabel('Open Rate')

plt.subplot(2, 3, 4)
sns.barplot(x='email_version', y='clicked', data=df_analysis)
plt.title('Click Rate by Email Version')
plt.ylabel('Click Rate')

plt.subplot(2, 3, 5)
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_open_rates = df_analysis.groupby('weekday')['opened'].mean().reindex(weekday_order)
weekday_open_rates.plot(kind='bar')
plt.title('Open Rate by Weekday')
plt.ylabel('Open Rate')

plt.subplot(2, 3, 6)
hour_click_rates = df_analysis.groupby('hour')['clicked'].mean()
hour_click_rates.plot(kind='line', marker='o')
plt.title('Click Rate by Hour of Day')
plt.xlabel('Hour')
plt.ylabel('Click Rate')

plt.tight_layout()
plt.savefig('email_campaign_eda.png')
plt.close()

In [29]:
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
country_click_rates = df_analysis.groupby('user_country')['clicked'].mean().sort_values(ascending=False)
country_click_rates.plot(kind='bar')
plt.title('Click Rate by Country')
plt.ylabel('Click Rate')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
df_analysis['purchase_category'] = pd.cut(df_analysis['user_past_purchases'],
                                         bins=[0, 1, 3, 5, 10, float('inf')],
                                         labels=['0', '1-3', '4-5', '6-10', '10+'])
purchase_click_rates = df_analysis.groupby('purchase_category')['clicked'].mean()
purchase_click_rates.plot(kind='bar')
plt.title('Click Rate by Past Purchases')
plt.ylabel('Click Rate')

plt.tight_layout()
plt.savefig('email_campaign_eda_2.png')
plt.close()


#**Segment Analysis**

In [30]:
df_analysis['hour_segment'] = pd.cut(df_analysis['hour'],
                                    bins=[0, 6, 12, 18, 24],
                                    labels=['Night (0-6)', 'Morning (6-12)', 'Afternoon (12-18)', 'Evening (18-24)'])

In [31]:
plt.figure(figsize=(10, 6))
heatmap_data = df_analysis.pivot_table(values='clicked', index='weekday',
                                       columns='hour_segment', aggfunc='mean')
sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu', fmt='.2%')
plt.title('Click Rate by Weekday and Time of Day')
plt.tight_layout()
plt.savefig('weekday_hour_heatmap.png')
plt.close()

In [32]:
plt.figure(figsize=(10, 6))
segment_data = df_analysis.pivot_table(values='clicked', index='email_version',
                                      columns='email_text', aggfunc='mean')
sns.heatmap(segment_data, annot=True, cmap='YlGnBu', fmt='.2%')
plt.title('Click Rate by Email Version and Text Length')
plt.tight_layout()
plt.savefig('version_text_heatmap.png')
plt.close()

# **Model Building**

In [33]:
X = df_analysis.drop(['email_id', 'opened', 'clicked', 'purchase_category', 'hour_segment'], axis=1)
y = df_analysis['clicked']  # Target is whether the link was clicked

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

"Defining categorical and numerical features"
categorical_features = ['email_text', 'email_version', 'weekday', 'user_country']
numerical_features = ['hour', 'user_past_purchases']


"Creating a column transformer for preprocessing"
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

In [36]:
log_reg_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])  #Logistic Regression

In [37]:
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])  #Random Forest

In [38]:
gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42))
])  #Gradient Boosting

In [40]:
models = {
    'Logistic Regression': log_reg_pipeline,
    'Random Forest': rf_pipeline,
    'Gradient Boosting': gb_pipeline
}

print("\nModel Cross-Validation Results:")
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    print(f"{name} - Mean ROC AUC: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")


Model Cross-Validation Results:
Logistic Regression - Mean ROC AUC: 0.7341 (±0.0127)
Random Forest - Mean ROC AUC: 0.5795 (±0.0139)
Gradient Boosting - Mean ROC AUC: 0.7408 (±0.0128)


In [42]:
# Based on the cross-validation results, tuned the Gradient Boosting model
param_grid = {
    'classifier__n_estimators': [100],
    'classifier__learning_rate': [0.1],
    'classifier__max_depth': [3]
}

grid_search = GridSearchCV(gb_pipeline, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("\nBest Hyperparameters:", grid_search.best_params_)
print("Best CV Score:", grid_search.best_score_)


Best Hyperparameters: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 100}
Best CV Score: 0.7407673832454572


In [43]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Calculating performance metrics
print("\nTest Set Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Test Set Performance:
Accuracy: 0.9787333333333333
ROC AUC: 0.7466435979452014

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     29364
           1       0.00      0.00      0.00       636

    accuracy                           0.98     30000
   macro avg       0.49      0.50      0.49     30000
weighted avg       0.96      0.98      0.97     30000



In [44]:
feature_names = []
ohe = best_model.named_steps['preprocessor'].transformers_[0][1]
cat_feature_names = list(ohe.get_feature_names_out(categorical_features))
feature_names.extend(cat_feature_names)
feature_names.extend(numerical_features)

# feature importances
importances = best_model.named_steps['classifier'].feature_importances_

# A DataFrame for feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Sorting by importance
feature_importance = feature_importance.sort_values('Importance', ascending=False).head(15)

# Plotting
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Top 15 Feature Importances')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

In [45]:
baseline_click_rate = y.mean()

#predicted probabilities for the entire dataset
df_analysis['click_probability'] = best_model.predict_proba(X)[:, 1]

#simulate targeting only users with high predicted click probability
threshold = df_analysis['click_probability'].quantile(0.7)  #Targeting top 30%
targeted_users = df_analysis[df_analysis['click_probability'] >= threshold]
expected_click_rate = targeted_users['clicked'].mean()

print("\nExpected Improvement:")
print(f"Baseline Click Rate: {baseline_click_rate:.4f}")
print(f"Expected Click Rate with Model: {expected_click_rate:.4f}")
print(f"Relative Improvement: {((expected_click_rate - baseline_click_rate) / baseline_click_rate * 100):.2f}%")



Expected Improvement:
Baseline Click Rate: 0.0212
Expected Click Rate with Model: 0.0463
Relative Improvement: 118.73%


In [46]:
def simulate_ab_test(baseline_rate, expected_rate, sample_size=5000, n_simulations=1000):
    """Simulate A/B test results"""
    p_values = []
    for _ in range(n_simulations):
        control = np.random.binomial(1, baseline_rate, sample_size)
        treatment = np.random.binomial(1, expected_rate, sample_size)

        # Calc p-value using chi-square test
        contingency_table = np.array([
            [sum(control), sample_size - sum(control)],
            [sum(treatment), sample_size - sum(treatment)]
        ])

        from scipy.stats import chi2_contingency
        _, p_value, _, _ = chi2_contingency(contingency_table)
        p_values.append(p_value)

    power = sum(np.array(p_values) < 0.05) / len(p_values)
    return power

power = simulate_ab_test(baseline_click_rate, expected_click_rate)
print(f"\nA/B Test Power (sample size 5000 per group): {power:.4f}")


A/B Test Power (sample size 5000 per group): 1.0000


In [49]:
def calculate_sample_size(p1, p2, power=0.8, alpha=0.05):
    """Calculate required sample size for A/B test"""
    #standard normal deviates for alpha and beta
    z_alpha = norm.ppf(1 - alpha/2)
    z_beta = norm.ppf(power)

    p_pooled = (p1 + p2) / 2

    #Calc  sample size per group
    n = ((z_alpha + z_beta)**2 * p_pooled * (1 - p_pooled)) / ((p1 - p2)**2)

    return int(np.ceil(n))

sample_size = calculate_sample_size(baseline_click_rate, expected_click_rate)
print(f"Required sample size per group for 80% power: {sample_size}")


Required sample size per group for 80% power: 405


In [51]:
print("\n=== SUMMARY OF FINDINGS ===")
print("1. Email Campaign Performance:")
print(f"   - Open Rate: {open_rate:.2f}%")
print(f"   - Click Rate: {click_rate:.2f}%")
print(f"   - Click-to-Open Rate: {click_to_open_rate:.2f}%")

print("\n3. Model Results:")
print(f"   - Best model: Gradient Boosting with AUC of {roc_auc_score(y_test, y_pred_proba):.4f}")
print(f"   - Expected improvement in click rate: {((expected_click_rate - baseline_click_rate) / baseline_click_rate * 100):.2f}%")

print("\n4. Recommendations:")
print("   - Implement personalized email targeting using the model")
print("   - Optimize sending times based on user segments")
print("   - Tailor email content length to user preferences")
print("   - Conduct an A/B test with sample size of at least", sample_size, "per group")
print("   - Develop specific strategies for high-value user segments")


=== SUMMARY OF FINDINGS ===
1. Email Campaign Performance:
   - Open Rate: 10.35%
   - Click Rate: 2.12%
   - Click-to-Open Rate: 20.48%

3. Model Results:
   - Best model: Gradient Boosting with AUC of 0.7466
   - Expected improvement in click rate: 118.73%

4. Recommendations:
   - Implement personalized email targeting using the model
   - Optimize sending times based on user segments
   - Tailor email content length to user preferences
   - Conduct an A/B test with sample size of at least 405 per group
   - Develop specific strategies for high-value user segments
